In [6]:
import keras
import numpy as np
import glob 
import os
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten, BatchNormalization
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
import random
import cv2
from keras import backend
from scipy.stats import norm
import imageio

In [7]:
#val_path = ['../../../../data/tfrecords/val/20200925/val_5000.tfrecords'
model_dir = '../../data/models/20201021_unrestr_gaussian_resampled/'
shard_path = '../../../data/commaai/training_files/unrestricted_gauss_dens_resampled/'
val_path = '../../data/commaai/test_files/'

In [8]:
c = 2000000
EPOCHS = 50
STEPS = c*EPOCHS

In [9]:
shard_files = glob.glob(os.path.join(shard_path, "*.tfrecords")) 
#random.shuffle(shard_files)
#shard_files = ['../data/commaai/train_files_from_bag_test/0_new2.tfrecords']

#val_files = glob.glob(os.path.join(val_path, "*.tfrecords")) 
val_files = ['../../data/commaai/test_files/_test.tfrecords']

In [12]:
def imgs_input_fn(filenames, perform_shuffle = True, repeat_count = EPOCHS, batch_size = 56): 
    
    # reads in single training example and returns it in a format that the estimator can
    # use
    def _parse_function(proto):
        # define your tfrecord again. Remember that you saved your image as a string.
        keys_to_features = {'image': tf.io.FixedLenFeature([], tf.string),
                            'label': tf.io.FixedLenFeature([], tf.float32),
                            'rows': tf.io.FixedLenFeature([], tf.int64),
                            'cols': tf.io.FixedLenFeature([], tf.int64),
                            'depth': tf.io.FixedLenFeature([], tf.int64),
                            'tr_label': tf.io.FixedLenFeature([], tf.float32)
                           }

        # Load one example
        parsed_example = tf.io.parse_single_example(proto, keys_to_features)

        image_shape = tf.stack([874, 1164, 4])
        image_raw = parsed_example['image']

        label = tf.cast(parsed_example['label'], tf.float32)
        tr_label = tf.cast(parsed_example['tr_label'], tf.float32)
        image = tf.io.decode_raw(image_raw, tf.uint8)
        image = tf.cast(image, tf.float32)

        image = tf.reshape(image, image_shape)
        image = tf.image.resize(image, [218, 291])[76:142, 45:245,0:3]
        image = image/255 
        
        return {'image':image}, tr_label
    
    dataset = tf.data.TFRecordDataset(filenames = filenames)
    dataset = dataset.map(_parse_function)
    if perform_shuffle:
        # Randomizes input using a window of 256 elements (read into memory)
        dataset = dataset.shuffle(buffer_size=256)
    dataset = dataset.repeat(repeat_count)  # Repeats dataset this # times
    dataset = dataset.batch(batch_size)  # Batch size to use
    iterator = tf.compat.v1.data.make_one_shot_iterator(dataset)
    batch_features, batch_labels = iterator.get_next()
    
    return batch_features, batch_labels


In [14]:
images, labels = imgs_input_fn(shard_path + '0.tfrecords')

In [ ]:
for file in shard_files:
    print('reading file ' + str(file))
    images, labels = imgs_input_fn(file)

In [ ]:
plt.imshow(np.array(images['image'][0]))

In [ ]:
plt.imshow(img[:,:,0:3])

In [ ]:
plt.imshow(img[:,:,3])

In [ ]:
img[:,:,3].tolist()